In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InR

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-03 17:03:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-12-03 17:03:00 (10.7 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark_Instrument").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [5]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

904765
904663
904663


In [25]:
from pyspark.sql.functions import to_date

review_id_table = df.select('review_id', 'customer_id', 'product_id', 'product_parent', to_date('review_date', "yyyy-MM-dd").alias('review_date'))
review_id_table.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100LMJJNWFDJK|   12078619|B00B054NVG|     963648561| 2014-10-24|
|R10106OQ9QP86T|   43421823|B004W4TYMA|     461512942| 2015-03-30|
|R101ER44F5J3EK|   49269789|B00AM966CU|     172551627| 2015-02-04|
|R1036AJM4DUM64|   17911630|B006X735PQ|     627151806| 2013-04-03|
|R103CWY3MPPRYB|   31063425|B00DY1F2CS|     554396934| 2015-08-02|
|R104VHJ7WLB4PP|    7146158|B00J0823FI|     647000350| 2015-06-11|
|R105A5ZRV1UHT1|   15499595|B0002H0FZ6|     497097576| 2014-10-18|
|R105RU48J2VL19|   17984606|B005DBF0AG|     990545730| 2015-06-15|
|R106LKOCWLJKP1|   14027720|B00558JGN4|     463070117| 2013-02-13|
|R106NRR8JSGUID|   20032109|B000VTMUQ2|     477030445| 2014-07-05|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [7]:
products = df.select('product_id', 'product_title')
products = products.dropDuplicates()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00ASHQQKI|SUPERNIGHT 5050 1...|
|B001BQ9HR4|Maxon AD-9 Analog...|
|B001B9JZ78|Clarke SBDC Penny...|
|B000BHJX96|Viva VLML5 Standa...|
|B0002E1TAY|Gibson Masterbuil...|
|B0056XAWOU|Griffin GC20007 D...|
|B00JJ18VIE|Reliable Hardware...|
|B0002H05BA|D'Addario EJ21 Ni...|
|B000WS1QC6|Yamaha PA130 120 ...|
|B0006GRRQI|Gator G-PG ACOUST...|
|B0006NMUJU|Shure Foam Windsc...|
|B005DJOI8I|Neewer 3.5mm Hand...|
|B006NH9OC4|Cakewalk SONAR X1...|
|B0046HADYQ|Shure EAC64BK 64 ...|
|B003B1PFWS|Planet Waves 1/8 ...|
|B00I81TNHO|Dean Markley 2034...|
|B000W348OO|Peavey Pvi 100 Dy...|
|B003AYI8H0|Fender Bushings V...|
|B0002KZQ7U|Behringer V-Amp P...|
|B009NO06JO|Yamaha THR AMPS T...|
+----------+--------------------+
only showing top 20 rows



In [8]:
customers = df.select('customer_id')
customers = customers.groupBy('customer_id').count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10256687|             3|
|   38115097|             1|
|   45891545|             5|
|   35646935|             1|
|    5373021|             1|
|    2533410|            11|
|   12886089|             1|
|   45599268|             1|
|   44267121|             8|
|   42984093|             1|
|   39095180|             1|
|   25232993|             1|
|   11263585|             5|
|   29660587|             3|
|   24652965|             3|
|   52600605|             2|
|   10012354|             2|
|   10208065|             2|
|   43381859|             1|
|   42208849|            11|
+-----------+--------------+
only showing top 20 rows



In [9]:
vine_table = df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100LMJJNWFDJK|          5|            0|          0|   N|
|R10106OQ9QP86T|          5|            0|          1|   N|
|R101ER44F5J3EK|          5|            0|          0|   N|
|R1036AJM4DUM64|          4|            0|          0|   N|
|R103CWY3MPPRYB|          2|            0|          0|   N|
|R104VHJ7WLB4PP|          5|            0|          0|   N|
|R105A5ZRV1UHT1|          5|            0|          0|   N|
|R105RU48J2VL19|          5|           12|         12|   N|
|R106LKOCWLJKP1|          1|            2|          8|   N|
|R106NRR8JSGUID|          5|            0|          0|   N|
|R106V7A3TQ944V|          5|            0|          0|   N|
|R109H6GZEOXX1S|          5|            2|          2|   N|
|R10ACINEOPG32H|          1|            0|          0|   N|
|R10BNDWVSS8YML|          5|            

#**RDS**

In [17]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<database>:5432/InstrumentDB"
config = {"user":"<user>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [18]:
# Write DataFrame to table

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [19]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [20]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [26]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)